# SAM: TP1 Accès aux données avec index 

Sujet pour étudiants

date de modification : 26/01/2023 16h

NOM: VIN

Prénom: Charles

Objectifs:
Savoir organiser des données en pages pour permettre de modifier un tuple en ne modifiant qu'une seule page.

Comprendre les méthodes d'accès suivantes :

*   Lecture séquentielle d'une fichier : "table access full"
*   Lecture d'un tuple dont on connait le rowid : "table access by index rowid"
*   Opération de sélection par lecture séquentielle et filtrage 

Comprendre les méthodes d'indexation :

*   Créer un index
*   Opération de Sélection par index et lecture par rowid

Mise à jour de données
*   Sélectionner un tuple et modifier un de ses attributs
*   Modifier l'index en conséquence lorsque l'attibut modifié est indexé

Persistence
*   Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement
*   Adapter en conséquence les opérations de modification de l'index


In [1]:
import os
import shutil as sh
import numpy as np
import random
from random import choice
from string import ascii_lowercase
import time

DATA = "data.csv"

# Générer un fichier

Création du fichier

In [2]:
# nb_lines = 5 * 1000 * 1000
nb_lines = 10000
nb_attributes = 7
longueur_attribut = 100
nb_valeurs_distinctes = nb_lines
try:
  open(DATA, "r")
except FileNotFoundError:
  # dure environ 40s pour 5M lignes
  # string_val = "".join(choice(ascii_lowercase) for i in range(longueur_attribut))
  long_string = ''.join('-' for i in range(longueur_attribut))

  # a=[np.random.randint(0, int(nb_lines/(10**i)), nb_lines) for i in range(nb_attributes)]

  # le premier attribut est unique
  a = [random.sample(range(nb_valeurs_distinctes), nb_lines)]

  # les attributs suivants ont des domaines plus petits
  for i in range(1, nb_attributes):
    nb_valeurs_distinctes = max(2, int(nb_valeurs_distinctes / 2))
    a.append(np.random.randint(0, nb_valeurs_distinctes, nb_lines))

  b = [ ','.join(map(lambda x: str(x), e)) + f",{long_string}\n" for e in zip(*a)]

  with open(DATA, "w") as f:
    f.write(''.join(b))

In [3]:
%%bash
echo "head : "
head -n 2 data.csv
echo "tail : "
tail -n 2 data.csv
echo "size (lines) :"
wc -l data.csv

head : 
5191,356,504,875,319,132,83,----------------------------------------------------------------------------------------------------
8636,1548,791,155,471,291,102,----------------------------------------------------------------------------------------------------
tail : 
7945,4972,410,1083,141,78,42,----------------------------------------------------------------------------------------------------
1802,1159,956,811,541,81,8,----------------------------------------------------------------------------------------------------
size (lines) :
10000 data.csv


# Lecture séquentielle

In [4]:
def filtrer_fichier(fichier, valeur_recherchee):
  with open(fichier, "r") as f:
    for i, line in enumerate(f):
      a = int(line.split(',')[0])
      if a == s :
        print(f"ligne {i} :", line.strip())


s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier(DATA, s)
print("done in", time.time() - t1, "s")

valeur recherchée : 530
ligne 3441 : 530,2486,1470,56,224,96,42,----------------------------------------------------------------------------------------------------
done in 0.01744675636291504 s


# Découper le fichier en pages

In [5]:
def page_dir_name(fichier):
  return fichier.split('.')[0] + "_pages"

def decoupe_fichier_en_pages(fichier, nb_tuple_par_page):
  page_dir = page_dir_name(fichier)
  print("pages dans :", page_dir)
  if(os.path.exists(page_dir)):
    sh.rmtree(page_dir)
  os.makedirs(page_dir, exist_ok=True)

  with open(fichier, "r") as f:
    p=0
    lines = []
    for i, line in enumerate(f):
      lines.append(line)
      if (i+1) % nb_tuple_par_page == 0:
        p += 1
        with open(page_dir + f"/page{p}", "w") as fp:
          fp.write(''.join(lines))
        lines = []
    if len(lines) > 0:
      p +=1
      with open(page_dir + f"/page{p}", "w") as fp:
          fp.write(''.join(lines))
    
    print("nb pages créées :", p)

decoupe_fichier_en_pages(DATA, nb_tuple_par_page=1000)

pages dans : data_pages
nb pages créées : 10


Afficher le nombre de tuples dans une page (pour quelques pages)

In [6]:
%%bash
wc -l data_pages/* | head -n 3

   1000 data_pages/page1
   1000 data_pages/page10
   1000 data_pages/page2


# Lecture séquentielle du fichier découpé en pages

In [7]:
def lecture_sequentielle_par_page(fichier):
#   # a faire : pour chaque page, lire ses lignes
#   # une ligne devient un tuple
#   # retourner un itérateur contenant le numéro de page, la position dans la page et le tuple
    page_dir = page_dir_name(fichier)
    nb_pages = len(os.listdir(page_dir))
    for page_nb in range(1, nb_pages+1):
        with open(page_dir + "/page" + str(page_nb), "r") as page:
            for i, line in enumerate(page):
                yield [page_nb, i, tuple(line.split(","))]


def filtrer_fichier_par_pages(fichier, valeur_recherchee):
    l = lecture_sequentielle_par_page(fichier)
    print('Research started')
    for page_nb, pos_page, line in l:
        if int(line[0]) == valeur_recherchee:
            print(f"Page n°{page_nb}, Line n°{pos_page} :", line)

#   # a faire pour chaque (numéro de page, position dans la page, tuple) obtnenu en invoquan la méthode ci dessus
#   # convertir le 1er attribut en un nombre l'afficher si il est egal à la valeur recherchee



s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier_par_pages(DATA, s)
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 7702
Research started
Page n°2, Line n°299 : ('7702', '505', '413', '924', '112', '183', '149', '----------------------------------------------------------------------------------------------------\n')
done in 0.03 s


# Lecture d'un tuple dans une page

In [8]:
def lecture_tuple(fichier, num_page, position):
    page_dir = page_dir_name(fichier)
    with open(page_dir + '/page' + str(num_page), 'r') as page:
        return page.readlines()[position]
lecture_tuple(DATA, 1, 21)

'9440,3586,1428,940,116,55,152,----------------------------------------------------------------------------------------------------\n'

# Créer un index

In [9]:
def creation_index_unique(fichier):
    index = {}
    page_dir = page_dir_name(fichier)
    nb_pages = len(os.listdir(page_dir))
    for page_nb in range(1, nb_pages+1):
        with open(page_dir + "/page" + str(page_nb), "r") as page:
            for i, line in enumerate(page):
                index[int(line.split(",")[0])] = (page_nb, i)
    return index
#    # la clé est la valeur du 1er attribut
#    # la valeur est un rowid composé de (page, position)



t1 = time.time()
index1 = creation_index_unique(DATA)
print("done in", round(time.time() - t1, 2), "s")

done in 0.07 s


# Accès par index

## Index unique scan
Accès pour rechercher les tuples dont le 1er attribut a une valeur donnée.

On peut supposer pour simplifier que l'attribut est unique

In [10]:
def selection_par_index(fichier, valeur_recherchee):
    index1 = creation_index_unique(DATA)
    return index1[valeur_recherchee]

In [11]:
selection_par_index(DATA, s)

(2, 299)

## Index range scan
Accès pour rechercher les tuples dont le 1er attribut a une valeur comprise dans une intervalle donné

In [12]:
def selection_par_index_plage(fichier, borne_inf, borne_sup):
    tuple_list = []
    index = creation_index_unique(DATA)
    for key in index.keys():
        if borne_inf < key and key < borne_sup:
            tuple_list.append(index[key])
    return tuple_list

In [13]:
selection_par_index_plage(DATA, 10, 15)

[(1, 815), (6, 108), (7, 128), (7, 946)]

# Mise à jour de données




## Sélectionner un tuple et modifier un de ses attributs



## Modifier l'index en conséquence lorsque l'attibut modifié est indexé


# Persistence



## Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement


## Adapter en conséquence les opérations de modification de l'index